In [1]:
from IPython.display import display, Markdown
from datetime import datetime
cur_datetime = datetime.now()
display(Markdown(f'# {cur_datetime.strftime("%d/%b/%Y %H:%M")}'))

# 26/Aug/2020 11:21

In [2]:
%%capture
# %matplotlib inline

# Dependences
from time import time
# import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import numpy as np

# Experiments
import run

In [3]:
# Run all experiments. Typical run duration for an Core-i3 laptop is about 2-3min.
# Tweak the prey_predator_abm/sim_params.py file if you want it to take longer (or not).
start_time = time()
experiments = run.run()
end_time = time()
print("Execution in {:.1f}s".format(end_time - start_time))


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: multi_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (147437, 3, 1, 3)
Execution Method: single_proc_exec
Execution Mode: single_threaded
Total execution time: 6.32s
Execution in 6.6s


In [4]:
experiments

,DAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep
0,5900000000000000000000,30000000000000000000,30000000000000000000,0,0,1,0,0
1,5905000000000000000000,29974673940746674209,30000000000000000000,0,0,1,1,1
2,5907000000000000000000,29964555511671268196,30000000000000000000,0,0,1,1,2
3,5905035238623641640532,29974555511671268196,30000000000000000000,0,0,1,1,3
4,5903071785798411448958,29984555511671268196,30000000000000000000,0,0,1,1,4
...,...,...,...,...,...,...,...,...
147433,654879499351386157772783,5092992005393848170161,3308419107140103200194,0,0,1,1,147433
147434,654816098175151717483841,5093486607320263785891,3308419107140103200194,0,0,1,1,147434
147435,655128708431980968543553,5091063416914910523856,3308419107140103200194,0,0,1,1,147435
147436,655274248273480968543553,5089936056577998314834,3308419107140103200194,0,0,1,1,147436
